In [1]:
import requests
import mysql.connector
import pandas as pd

In [2]:
AAPL = pd.read_csv('AAPL.csv')

In [3]:
Comps = ["MSFT", "AAPL", "NVDA", "AVGO", 
        #  "BRK-B", "JPM",
         "LLY", "UNH", "AMZN", "GE", "GOOG", "WMT", "XOM", "LIN",]

In [4]:
t = []
for i in range(len(Comps)):
    t.append(pd.read_csv(f'../data/sentiment_scores/{Comps[i]}.csv'))
    t[i]['Unnamed: 0']=Comps[i]

In [5]:
df = t[0]
for i in range(1,len(Comps)):
    df = pd.concat([df,t[i]],axis = 0)
df

,Unnamed: 0,time,scores
0,MSFT,20220301T203854,0.031452
1,MSFT,20220302T030642,-0.145930
2,MSFT,20220302T080000,-0.520567
3,MSFT,20220302T080000,-0.162332
4,MSFT,20220302T093019,-0.093892
...,...,...,...
641,LIN,20240327T113800,0.296809
642,LIN,20240327T131333,0.228489
643,LIN,20240328T080000,0.351409
644,LIN,20240328T201500,0.370743


In [6]:
df.rename(columns={'Unnamed: 0':'Symbol','time':'Date','scores':'Sentiment_scores'},inplace = True)

In [7]:
df

,Symbol,Date,Sentiment_scores
0,MSFT,20220301T203854,0.031452
1,MSFT,20220302T030642,-0.145930
2,MSFT,20220302T080000,-0.520567
3,MSFT,20220302T080000,-0.162332
4,MSFT,20220302T093019,-0.093892
...,...,...,...
641,LIN,20240327T113800,0.296809
642,LIN,20240327T131333,0.228489
643,LIN,20240328T080000,0.351409
644,LIN,20240328T201500,0.370743


In [35]:
def create_mysql_table():
    mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="root",
        database="qf",
        port = "3307"
    )
    mycursor = mydb.cursor()
    mycursor.execute(
        '''CREATE TABLE sentiment (
            id INT AUTO_INCREMENT PRIMARY KEY, 
            symbol VARCHAR(10), 
            date VARCHAR(255),
            sentiment_Scores FLOAT
            )
        '''
        )
    mydb.commit()
    mydb.close()

def insert_mysql_table(data):
    mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="root",
        database="qf",
        port = "3307"
    )

    mycursor = mydb.cursor()

    # sql = "INSERT INTO sentiment (symbol, date, sentiment_scores) VALUES (%s, %s, %s)"
    # val = (data['Symbol'], data['Date'], data['Sentiment_scores'])
    
    # mycursor.execute(sql, val)

    for index, row in data.iterrows():
        sql = "INSERT INTO sentiment (Symbol, Date, Sentiment_scores) VALUES (%s, %s, %s)"
        val = (row['Symbol'], row['Date'], row['Sentiment_scores'])
        mycursor.execute(sql, val)
        
    mydb.commit()
    mydb.close()

# def main():
#     records = get_month_news_for_companies(Comps, 2022, 2023)

#     # create_mysql_table()

#     for record in records:
#         for data in record:
#             insert_mysql_table(data)


In [12]:
create_mysql_table()

In [36]:
insert_mysql_table(df)